In [7]:
import numpy
import pandas
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU

import yfinance
tsla_data = yfinance.download("TSLA", start="2020-04-11", end="2025-04-11")

#computing Shock variable +/- 20% change in volume

#computing shocks - 20%+ change in volume
tsla_volume_lag = tsla_data["Volume"].shift(1)
tsla_data["Shock"] = (abs(tsla_volume_lag.values - tsla_data["Volume"].values) > 0.2 * tsla_data["Volume"].values)

# Show count of shocks
print(tsla_data["Shock"].value_counts())

#plotting daily Tesla stock closing prices
train = tsla_data[tsla_data.index < pandas.to_datetime("2025-01-02", format='%Y-%m-%d')]
test = tsla_data[tsla_data.index >= pandas.to_datetime("2025-01-02", format='%Y-%m-%d')]

#splitting into training and testing sets
train = tsla_data[tsla_data.index < pandas.to_datetime("2025-01-02", format='%Y-%m-%d')]
test = tsla_data[tsla_data.index >= pandas.to_datetime("2025-01-02", format='%Y-%m-%d')]

train_set=train.loc[:, "Shock"].values
test_set=test.loc[:, "Shock"].values

#splitting training data into samples
nsteps = 60

def split_sequence(sequence):
    x, y = list(), list()
    for i in range(len(sequence)):
        end_i = i + nsteps
        if end_i > len(sequence)-1:
            break
        seq_x, seq_y=sequence[i:end_i], sequence[end_i]
        x.append(seq_x)
        y.append(seq_y)
    return numpy.array(x), numpy.array(y)

train_x, train_y = split_sequence(train_set)

[*********************100%***********************]  1 of 1 completed

Shock
False    732
True     525
Name: count, dtype: int64


In [8]:
###############################################################
#FITTING LSTM MODEL
###############################################################
#reshaping train_x
features = 1
train_x=train_x.reshape(train_x.shape[0],train_x.shape[1],features)

#specifying model architecture
model_lstm = Sequential()
model_lstm.add(LSTM(units=6, activation="sigmoid", input_shape=(nsteps, features)))
model_lstm.add(Dense(units=1))

# Compiling the model
model_lstm.compile(loss="binary_crossentropy")
model_lstm.fit(train_x, train_y, epochs=5, batch_size=32)
inputs = tsla_data.loc[:,"Shock"][len(tsla_data.loc[:,"Shock"])-len(test_set)-nsteps :].values

#splitting into samples
test_x, test_y = split_sequence(inputs)

#reshaping
test_x = test_x.reshape(test_x.shape[0], test_x.shape[1], features)

#predicting for testing data
pred_prob=model_lstm.predict(test_x)

cutoff=[]
accuracy=[]
for i in range(99):
    tp=0
    tn=0
    cutoff.append(0.01*(i+1))
    for sub1, sub2 in zip(pred_prob, test_y):
        tp_ind=1 if (sub1>0.01*(i+1) and sub2==1) else 0
        tn_ind=1 if (sub1<0.01*(i+1) and sub2==0) else 0
        tp+=tp_ind
        tn+=tn_ind
    accuracy_i=(tp+tn)/len(pred_prob)
    accuracy.append(accuracy_i)
    
df=pandas.DataFrame({'accuracy': accuracy,'cut-off': cutoff})
max_accuracy=max(accuracy)
optimal=df[df['accuracy']==max_accuracy]
print(optimal)

Epoch 1/5


C:\Users\000110888\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 6.4864
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6.2828
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6.5456
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 6.4850
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 6.7410
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/stepWARNING:tensorflow:6 out of the last 9 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000001304B2A7E20> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and 

In [9]:
###############################################################
#FITTING GRU MODEL
###############################################################
#specifying model architecture
model_gru = Sequential()
model_gru.add(GRU(units=6, activation="sigmoid", input_shape=(nsteps, features)))
model_gru.add(Dense(units=1))

# Compiling the model
model_gru.compile(loss="binary_crossentropy")
model_gru.fit(train_x, train_y, epochs=5, batch_size=32)

#predicting for testing data
pred_prob=model_gru.predict(test_x)

cutoff=[]
accuracy=[]
for i in range(99):
    tp=0
    tn=0
    cutoff.append(0.01*(i+1))
    for sub1, sub2 in zip(pred_prob, test_y):
        tp_ind=1 if (sub1>0.01*(i+1) and sub2==1) else 0
        tn_ind=1 if (sub1<0.01*(i+1) and sub2==0) else 0
        tp+=tp_ind
        tn+=tn_ind
        
    accuracy_i=(tp+tn)/len(pred_prob)
    accuracy.append(accuracy_i)
    
df=pandas.DataFrame({'accuracy': accuracy,'cut-off': cutoff})
max_accuracy=max(accuracy)
optimal=df[df['accuracy']==max_accuracy]
print(optimal)

Epoch 1/5


C:\Users\000110888\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


36/36 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.7078
Epoch 2/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6777
Epoch 3/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6731
Epoch 4/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6722
Epoch 5/5
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.6728
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
    accuracy  cut-off
42  0.573529     0.43
43  0.573529     0.44
44  0.573529     0.45
45  0.573529     0.46
46  0.573529     0.47
47  0.573529     0.48
48  0.573529     0.49
49  0.573529     0.50
50  0.573529     0.51
51  0.573529     0.52
52  0.573529     0.53
53  0.573529     0.54
54  0.573529     0.55
55  0.573529     0.56
56  0.573529     0.57
57  0.573529     0.58
58  0.573529     0.59
59  0.573529     0.60
60  0.573529     0.61
61  0.573529     0.62
62  0.573529     0.63
63  0.573529     0.64
64  0.573529     0.65
65  0.573529     0.66
66  0.573529     0.67
67  0.573529     0.68
68  0.573529     0.69
69  0.573529     0.70
70  0.57